### IPython notebook for generating custom train/test split. If possible, use test split included in data_splits directory for consistency. 
#### Requires hardcoded paths to 50% clusters, as well as SwissProt fasta file. Uniref50 clusters filtered to Swissprot available on request. 

In [3]:
import numpy as np
import scipy
import pandas as pd
import random

def nested_len(cluster_list):
    return sum(len(x) for x in cluster_list)

def JS_divergence(training_counts, testing_counts):
    P = training_counts / np.linalg.norm(training_counts, ord=1)
    Q = testing_counts / np.linalg.norm(testing_counts, ord=1)
    M = 0.5 * (P + Q)
    return 0.5 * (scipy.stats.entropy(P, M) + scipy.stats.entropy(Q, M))

def load_clusters(clusters_path, protein_ids):
    cluster_list = []
    protein_count = 0
    reader = pd.read_csv(clusters_path, sep='\t', lineterminator='\n', chunksize = 10000)
    chunk_count = 0
    for chunk in reader:
        chunk_count += 1
        column = chunk["Cluster members"]
        for id_list in column:
            unfiltered_cluster = id_list.split("; ")
            cluster = []
            for prot_id in unfiltered_cluster:
                if(prot_id in protein_ids):
                    cluster.append(prot_id)
            if(len(cluster) > 0):
                cluster_list.append(cluster)
    return cluster_list

def build_random_split(cluster_list, train_fraction=0.9):
    protein_count = nested_len(cluster_list)
    random.shuffle(cluster_list)
    i = 0
    training_size = 0
    training_set = []
    while (training_size < train_fraction*protein_count):
        cluster = cluster_list[i]
        training_set.extend(cluster)
        training_size += len(cluster)
        i += 1
    testing_set = []
    while (i < len(cluster_list)):
        cluster = cluster_list[i]
        testing_set.extend(cluster)
        i += 1
    return training_set, testing_set

def build_random_cluster_split(cluster_list, train_fraction=0.9):
    protein_count = nested_len(cluster_list)
    random.shuffle(cluster_list)
    i = 0
    training_size = 0
    training_set = []
    while (training_size < train_fraction*protein_count):
        cluster = cluster_list[i]
        training_set.append(cluster)
        training_size += len(cluster)
        i += 1
    testing_set = []
    while (i < len(cluster_list)):
        cluster = cluster_list[i]
        testing_set.append(cluster)
        i += 1
    return training_set, testing_set

def enforce_threshold(annotation_count_dict, namespace, threshold):
    term_list = [] 
    for term, count in annotation_count_dict.items():
        if(godag[term].namespace == namespace or namespace == "all"):
            term_list.append((count, term))
    term_list.sort(reverse=True)
    i = 0
    while i < len(term_list) and term_list[i] >= threshold:
        i += 1
    return [x[1] for x in term_list[:i]]

In [2]:
import gzip
from Bio import SeqIO
sequences = []
prot_ids = []
with gzip.open("../data/swissprot/uniprot_sprot.fasta.gz", 'rt') as fp:
    seqs = SeqIO.parse(fp, 'fasta')
    count = 0
    for rec in seqs:
        seq_id = rec.id
        if('|' in seq_id):
            seq_id = rec.id.split('|')[1]
        seq = rec.seq
        sequences.append(str(seq.lower()))
        prot_ids.append(seq_id)
        if(count % 10000 == 0):
            print(f"{count} proteins processed")
        count += 1

0 proteins processed
10000 proteins processed
20000 proteins processed
30000 proteins processed
40000 proteins processed
50000 proteins processed
60000 proteins processed
70000 proteins processed
80000 proteins processed
90000 proteins processed
100000 proteins processed
110000 proteins processed
120000 proteins processed
130000 proteins processed
140000 proteins processed
150000 proteins processed
160000 proteins processed
170000 proteins processed
180000 proteins processed
190000 proteins processed
200000 proteins processed
210000 proteins processed
220000 proteins processed
230000 proteins processed
240000 proteins processed
250000 proteins processed
260000 proteins processed
270000 proteins processed
280000 proteins processed
290000 proteins processed
300000 proteins processed
310000 proteins processed
320000 proteins processed
330000 proteins processed
340000 proteins processed
350000 proteins processed
360000 proteins processed
370000 proteins processed
380000 proteins processed


In [3]:
clusters = load_clusters("../data/uniref-reviewed+identity_0.5.tab", set(prot_ids))

In [4]:
def build_random_cluster_split(cluster_list, train_fraction=0.7, val_fraction=0.15):
    protein_count = nested_len(cluster_list)
    random.shuffle(cluster_list)
    i = 0
    training_size = 0
    training_set = []
    while (training_size < train_fraction*protein_count):
        cluster = cluster_list[i]
        training_set.extend(cluster)
        training_size += len(cluster)
        i += 1
    validation_size = 0
    validation_set = []
    while (validation_size < val_fraction*protein_count):
        cluster = cluster_list[i]
        validation_set.extend(cluster)
        validation_size += len(cluster)
        i += 1
    testing_set = []
    while (i < len(cluster_list)):
        cluster = cluster_list[i]
        testing_set.extend(cluster)
        i += 1
    return training_set, validation_set, testing_set

training_set, validation_set, testing_set = build_random_cluster_split(clusters, train_fraction=0.7, val_fraction=0.15)
splits = (training_set, validation_set, testing_set)

In [9]:
sts = ["training", "validation", "testing"]
for ttt, ttt_set in zip(sts, splits):
        with open(f"../data/test_split_bootstrap2/test_split/{ttt}_ids.txt", "w") as f:
            f.writelines([x+"\n" for x in ttt_set])